# pandas2
- input/output
- pivot

## 1. input/output
- csv, excel, txt등의 파일을 읽거나 저장(엑셀과 csv의 활용이 압도적으로 많다)
- excel의 io를 위해서는 xlrd, openpyxl, xlsxwriter 등의 패키지가 필요하다.(없으면 아래와 같이 설치)
    - pip
        - `$ pip install xlrd openpyxl xlsxwriter`
    - conda
        - `$ conda install -c anaconda xlrd openpyxl xlsxwriter`

In [1]:
# 이전에 저장한 pickle파일 로드
import pandas as pd
import numpy as np
import pickle
with open("data.pkl", "rb") as f:
    result = pickle.load(f)

In [2]:
result.tail(3)

,ID,Name,Age,Money
5,6,Alan,39,5000
6,7,Adam,20,0
7,8,Alvin,33,28000


In [3]:
# dataframe을 csv로 저장
result.to_csv('foo.csv', index=False)

In [4]:
# load
df = pd.read_csv('foo.csv')
df.tail(3)

,ID,Name,Age,Money
5,6,Alan,39,5000
6,7,Adam,20,0
7,8,Alvin,33,28000


In [5]:
# excel로 저장하기
# encoding='utf-8-sig' 윈도우에서 한글 안깨지도록 하는 encoding
df.to_excel('foo.xlsx', sheet_name='Sheet1', engine='xlsxwriter', encoding='utf-8-sig')

In [6]:
# load
df = pd.read_excel('foo.xlsx', 'Sheet1')
df.tail(3)

,ID,Name,Age,Money
5,6,Alan,39,5000
6,7,Adam,20,0
7,8,Alvin,33,28000


## 2. pivot
- 데이터 프레임의 컬럼 데이터에서 index, columns, values를 선택해서 데이터 프레임을 만드는 방법
- `df.pivot(index, columns, values)`

### Index

1. Groupby와 Pivot을 이용

2. Pivot Table 을 이용

### * 드디어 예제다운 예제의 시작
- https://www.kaggle.com/

### * Kaggle
- 전세계 데이터 사이언티스트들의 경합소
- 데이터 사이언티스트들은 항상 데이터에 목마르다
- 기업은 데이터가 존시나 많지만 이걸 어떻게 분석해야할지 모른다
- 짜잔, Kaggle 사이트에 기업이 상금과 함께 데이터를 오픈하면, 전세계에서 날고 기는 데이터 사이언티스트들이 싹다 몰려들어 문제를 해결!
- 기업은 문제해결, 데이터 사이언티스트들은 상금받고 그렇게 원하던 데이터 분석도 하고!
- Competition이 끝난 주제는 상금을 받지는 못하지만, 그 데이터는 오픈되어있고 다른 사람들의 소스코드도 참고할 수 있어 이 곳만 잘 활용해도 스펙업 가능

### 거기서도 basic of basic인 titanic 데이터를 받아볼 예정
- titanic 침몰 사건때 승선했던 모든 사람들의 정보와 결론적으로 생존했는지 사망했는지를 기록한 데이터
- download : https://www.kaggle.com/c/titanic/data
- Survived - 0:no, 1:yes

In [7]:
# titanic data read
titanic = pd.read_csv("titanic/train.csv")
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### 1. Groupby와 Pivot을 이용

#### 1.1 성별, 좌석등급에 따른 데이터의 수를 나타내는 데이터 프레임

groupby로 성별과 좌성등급의 중복 데이터를 합쳐주고 size 함수로 데이터의 갯수 컬럼(Counts)를 추가

In [8]:
titanic_df1 = titanic.groupby(["Sex", "Pclass"]).size().reset_index(name="Counts")
titanic_df1

,Sex,Pclass,Counts
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


pivot을 이용하여 index, column, value 데이터를 설정

In [9]:
titanic_df2 = titanic_df1.pivot("Sex", "Pclass", "Counts")
titanic_df2

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


#### 1.2 성별에 따른 생존자 수를 나타내는 데이터 프레임

groupby로 성별과 생존의 중복 데이터를 합쳐주고 size 함수로 데이터의 갯수 컬럼(Counts)를 추가

In [10]:
df2 = titanic.groupby(["Sex", "Survived"]).size().reset_index(name="Counts")
df2

,Sex,Survived,Counts
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109


pivot을 이용하여 index, column, value 데이터를 설정

In [11]:
df2.pivot("Sex", "Survived", "Counts")

Survived,0,1
Sex,,
female,81,233
male,468,109


#### 1.3 객실등급에 따른 생존자수를 나타내는 데이터 프레임

groupby로 객실등급과 생존의 중복 데이터를 합쳐주고 size 함수로 데이터의 갯수 컬럼(Counts)를 추가

In [12]:
df3 = titanic.groupby(["Pclass", "Survived"]).size().reset_index(name="Counts")
df3

,Pclass,Survived,Counts
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,372
5,3,1,119


pivot을 이용하여 index, column, value 데이터를 설정

In [13]:
result = df3.pivot("Pclass", "Survived", "Counts")
result

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


### 2. Pivot Table 을 이용
- `pivot_table(values, index, columns, aggfunc)`
- fill_value : NaN 데이터를 우리가 설정한 데이터로 치환해주는 파라미터 입니다.
- dropna(True) : NaN 데이터 컬럼을 놔둘지 제거할지 결정할때 사용됩니다.

데이터의 수를 나타내주기 위해서 Counts 컬럼을 추가

In [14]:
titanic["Counts"] = 1
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Counts
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,1
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,1


#### 2.1 성별, 좌성등급에 따른 데이터의 수를 나타내는 데이터 프레임

In [15]:
titanic.pivot_table("Counts", ["Sex", "Survived"], ["Pclass"], aggfunc=np.sum)

Pclass            1   2    3
Sex    Survived             
female 0          3   6   72
       1         91  70   72
male   0         77  91  300
       1         45  17   47

#### 2.2 성별에 따른 생존자 수를 나타내는 데이터 프레임

In [16]:
titanic.pivot_table("Counts", ["Sex"], ["Survived"], aggfunc=np.sum)

Survived,0,1
Sex,,
female,81,233
male,468,109


#### 2.3 객실등급에 따른 생존자수를 나타내는 데이터 프레임

In [17]:
titanic.pivot_table("Counts", ["Pclass"], ["Survived"], aggfunc=np.sum)

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


#### 2.4 여러개의 index 데이터로 pivot

In [18]:
result = titanic.pivot_table("Counts", ["Sex", "Pclass"], [
                             "Survived"], aggfunc=np.sum)
result

Survived         0   1
Sex    Pclass         
female 1         3  91
       2         6  70
       3        72  72
male   1        77  45
       2        91  17
       3       300  47

#### 2.5 성별에 따른 생존자 Total 컬럼, 로우 추가 및 삭제

In [19]:
df = titanic.pivot_table("Counts", ["Survived"], ["Sex"], aggfunc=np.sum)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


total 데이터 추가

In [20]:
df["female"]

Survived
0     81
1    233
Name: female, dtype: int64

In [21]:
df["male"]

Survived
0    468
1    109
Name: male, dtype: int64

In [22]:
df["female"] + df["male"]

Survived
0    549
1    342
dtype: int64

In [23]:
df["total"] = df["female"] + df["male"]
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342


In [24]:
df.loc[0]

Sex
female     81
male      468
total     549
Name: 0, dtype: int64

In [25]:
df.loc["total"] = df.loc[0] + df.loc[1]
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342
total,314,577,891


데이터 삭제 - drop
- axis = 0 : row 를 삭제합니다.

In [26]:
df.drop("total", inplace=True)
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342


axis = 1 : column 를 삭제합니다.

In [27]:
df.drop("total", axis=1, inplace=True)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


#### 2.6 Pivot Table 에서 NaN에 대한 처리

In [28]:
df = titanic.pivot_table("Counts", ["Survived"], [
                         "Parch", "Pclass"], aggfunc=np.sum)
df

Parch        0                  1                 2                3       \
Pclass       1     2      3     1     2     3     1     2     3    2    3   
Survived                                                                    
0         64.0  86.0  295.0  10.0   8.0  35.0   5.0   3.0  32.0  NaN  2.0   
1         99.0  48.0   86.0  21.0  24.0  20.0  16.0  13.0  11.0  2.0  1.0   

Parch       4         5    6  
Pclass      1    3    3    3  
Survived                      
0         1.0  3.0  4.0  1.0  
1         NaN  NaN  1.0  NaN

fill_value : 데이터가 없을때 채워주는 파라미터

In [29]:
df = titanic.pivot_table("Counts", ["Survived"], ["Parch", "Pclass"],
                         aggfunc=np.sum, fill_value=0)
df

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

dropna = False : 없는 컬럼이나 로우를 드랍하지 않습니다.

In [30]:
df = titanic.pivot_table("Counts", ["Survived"], ["Parch", "Pclass"],
                         aggfunc=np.sum, dropna=False, fill_value=0)
df

Parch      0            1           2          3 ...     4        5        6  \
Pclass     1   2    3   1   2   3   1   2   3  1 ...  3  1  2  3  1  2  3  1   
Survived                                         ...                           
0         64  86  295  10   8  35   5   3  32  0 ...  2  1  0  3  0  0  4  0   
1         99  48   86  21  24  20  16  13  11  0 ...  1  0  0  0  0  0  1  0   

Parch           
Pclass    2  3  
Survived        
0         0  1  
1         0  0  

[2 rows x 21 columns]

pandas에서 display option 설정

In [31]:
pd.options.display.max_rows, pd.options.display.max_columns

(60, 20)

In [32]:
pd.options.display.max_columns = 21
df

Parch      0            1           2          3        4        5        6  \
Pclass     1   2    3   1   2   3   1   2   3  1  2  3  1  2  3  1  2  3  1   
Survived                                                                      
0         64  86  295  10   8  35   5   3  32  0  0  2  1  0  3  0  0  4  0   
1         99  48   86  21  24  20  16  13  11  0  2  1  0  0  0  0  0  1  0   

Parch           
Pclass    2  3  
Survived        
0         0  1  
1         0  0

In [33]:
pd.pivot_table

<function pandas.core.reshape.pivot.pivot_table>

### 데이터 처리
- 불필요한 feature 데이터 제거
- nan 데이터가 있는 레코드 제거
- 원핫인코딩
- 연령대 컬럼을 만들기
- 20대이상은 성인이라는 컬럼을 만들어 0, 1(이산데이터)를 추가

In [34]:
# 데이터 로드
titanic.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Counts
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,1
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,1


In [35]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Counts'],
      dtype='object')

In [36]:
# 필요한 컬럼데이터만 필터링
df = titanic[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
df.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
889,1,1,male,26.0,0,0,30.00,C
890,0,3,male,32.0,0,0,7.75,Q


In [37]:
# 필요없는 컬럼 데이터 제거 : drop
df = titanic.drop(columns=["PassengerId", "Name", "Ticket", "Cabin", "Counts"])
df.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
889,1,1,male,26.0,0,0,30.00,C
890,0,3,male,32.0,0,0,7.75,Q


In [38]:
df.head(6)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q


In [39]:
# nan 데이터가 있는 레코드 제거
result = df[df.notna().all(axis=1)].reset_index(drop=True)
result.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
710,1,1,male,26.0,0,0,30.00,C
711,0,3,male,32.0,0,0,7.75,Q


In [40]:
result["Sex"].unique()

array(['male', 'female'], dtype=object)

In [41]:
# 성별 feature 원핫 인코딩 : apply
result["male"] = result["Sex"].apply(lambda data: 1 if data == "male" else 0)
result["female"] = result["Sex"].apply(lambda data: 1 if data == "female" else 0)
result.tail()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,male,female
707,0,3,female,39.0,0,5,29.125,Q,0,1
708,0,2,male,27.0,0,0,13.000,S,1,0
709,1,1,female,19.0,0,0,30.000,S,0,1
710,1,1,male,26.0,0,0,30.000,C,1,0
711,0,3,male,32.0,0,0,7.750,Q,1,0


In [42]:
result = result.drop(columns=["male", "female"])

In [43]:
result.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
710,1,1,male,26.0,0,0,30.00,C
711,0,3,male,32.0,0,0,7.75,Q


In [44]:
result["Sex"][:5]

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [45]:
# pandas의 함수를 이용해서 원핫 인코딩
onehot_sex = pd.get_dummies(result["Sex"])
onehot_sex.tail()

,female,male
707,1,0
708,0,1
709,1,0
710,0,1
711,0,1


In [46]:
# 컬럼을 추가
result = pd.concat([result, onehot_sex], axis=1)
result.tail(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,female,male
710,1,1,male,26.0,0,0,30.00,C,0,1
711,0,3,male,32.0,0,0,7.75,Q,0,1


In [47]:
# 성별 컬럼 제거
result.drop(columns=["Sex"], inplace=True)

In [48]:
result.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,female,male
710,1,1,26.0,0,0,30.00,C,0,1
711,0,3,32.0,0,0,7.75,Q,0,1


In [49]:
# Embarked 컬럼을 원핫인코딩 하기
onehot_embarked = pd.get_dummies(result["Embarked"])
result = pd.concat([result, onehot_embarked], axis=1)
result.drop(columns=["Embarked"], axis=1, inplace=True)
result.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
710,1,1,26.0,0,0,30.00,0,1,1,0,0
711,0,3,32.0,0,0,7.75,0,1,0,1,0


In [50]:
# 연령대 컬럼을 만들기
result["Ages"] = ((result["Age"] // 10) * 10).astype("int")
result.tail(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Ages
710,1,1,26.0,0,0,30.00,0,1,1,0,0,20
711,0,3,32.0,0,0,7.75,0,1,0,1,0,30


In [51]:
# 20세 이상인 경우에는 Adult 컬럼을 만들어 1을 데이터로 추가
result["Adult"] = 0
result.tail(5)

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Ages,Adult
707,0,3,39.0,0,5,29.125,1,0,0,1,0,30,0
708,0,2,27.0,0,0,13.000,0,1,0,0,1,20,0
709,1,1,19.0,0,0,30.000,1,0,0,0,1,10,0
710,1,1,26.0,0,0,30.000,0,1,1,0,0,20,0
711,0,3,32.0,0,0,7.750,0,1,0,1,0,30,0


In [52]:
is_adult = result["Ages"] >= 20
is_adult[-5:]

707     True
708     True
709    False
710     True
711     True
Name: Ages, dtype: bool

In [53]:
result.loc[is_adult, "Adult"] = 1
result.tail()

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,Ages,Adult
707,0,3,39.0,0,5,29.125,1,0,0,1,0,30,1
708,0,2,27.0,0,0,13.000,0,1,0,0,1,20,1
709,1,1,19.0,0,0,30.000,1,0,0,0,1,10,0
710,1,1,26.0,0,0,30.000,0,1,1,0,0,20,1
711,0,3,32.0,0,0,7.750,0,1,0,1,0,30,1
